# どこかの日本企業の決算資料を用いてRAGを構築

In [5]:
from dotenv import load_dotenv
import os

In [6]:
# 環境変数読み込み
load_dotenv()

True

## 1. 決算資料(pdf)の読み込みと文章のチャンク化

In [3]:
from langchain_community.document_loaders import PyPDFLoader

# pdfのload
file_path = "./data/ricoh_finance_report.pdf"
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [5]:
len(pages)

32

In [7]:
pages[10].page_content

'© Ricoh Aug 6, 2024 1014 37 51 232 \n37 19 35 56 264 \n41 \n33% -4% 9% 14% 10%ITインフラ ITサービス AS BPS WEFY2023 Q1 FY2024 Q1373 417 \nFY2023 Q1 FY2024 Q1\nリコーデジタルサービス オフィスサービス概況:米州\n前年同期比\nFY2024 Q1 \n+11.8%\n+44  売上高 (億円)\n\uf06cBPSは前年並み\n\uf0fcオフサイト中心に好調\n\uf0fc業務効率化とプライシングコントロールに取組み収益性改善\n\uf06cワークプレイスエクスペリエンス（WE*2）、一部案件の期ずれ\n\uf0fcCenero*1とのシナジー拡大、米州既存顧客への導入すすむ\n\uf06cFY2023 Q4に事業の選択と集中に伴いeDiscovery事業売却分野別動向 (億円)\n前年同期比\n（除為替影響） (-16%) (-4%) (+0%) (-3%) (17%)*2一部商談の期ズレ等により弱含み。事業売却の一過性影響\n*1 買収企業の位置付けはP.24を参照\n*2 Workplace Experience/ ワークプレイスエクスペリエンス:コミュニケーションサービスより名称変更\n\uf06c米州のオフィスサービスです。\n\uf06c米州のBPSは、オフサイト中心に好調に推移しています。\n\uf06cなお、2023 年度までコミュニケーションサービスとしていた分野の名称を、より適切な表現にすべく 2024年度\nからワークプレイスエクスペリエンスと名称を変更しました。\n\uf06cこのワークプレイスエクスペリエンスでは、買収したCeneroが、大きなシナジーを生んでいます。\n第１四半期において、計画していた一部大型案件の期ずれが発生しているものの、順調に進展しています。\n\uf06c「事業の選択と集中」に伴い、3月に、主に訴訟のプロセスを支援する事業のeDiscoveryの売却をしたこと\nが業績（売上の減少）に影響しています。'

In [14]:
# チャンク化
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "。", "、", "\n", "！", "？", "；", "：", "「", "」", "『", "』", "（", "）", "・", "…", "　"],
)
all_splits = text_splitter.split_documents(pages)

In [15]:
len(all_splits)

34

In [16]:
for chunk in all_splits:
    print(chunk.page_content)
    print("-----------------")

FY2024 Q1 決算概要
2024年8月 6日
株式会社リコー
-----------------
© Ricoh Aug 6, 2024 1本資料に関する注意事項
本資料に記載されている、リコー (以下、当社) の現在の計画、見通し、戦略などのうち、歴史的事実でないものは、将来の業績に関する
見通しであり、現在入手可能な情報から得られた当社の経営者の判断に基づいております。 従って、実際の業績は異なる結果となる
場合がありますので、これらの業績見通しにのみ全面的に依拠なさらないようお願い致します。
実際の業績に影響を与えうる重要な要素には、 a) 当社の事業領域を取り巻く経済情勢、景気動向、 b) 為替レートの変動、 c) 当社
の事業領域に関連して発生する急速な技術革新、および d) 激しい競争にさらされた市場の中で、顧客に受け入れられる製品・サービスを
当社が設計・開発・生産し続ける能力、などが含まれます。ただし、業績に影響を与えうる要素はこれらに限定されるものではありません。
(参照:「事業等のリスク」 https://jp.ricoh.com/IR/risk.html)
本資料に他の会社・機関等の名称が掲載されている場合といえども、これらの会社・機関等の利用を当社が推奨するものではありません。
本資料に掲載されている情報は、投資勧誘を目的にしたものではありません。投資に関するご決定は、ご自身のご判断において行うよう
お願い致します。
本資料における年号の表記: 4月から始まる会計年度の表記としております。
(例) FY2024 (2024 年度):2024年4月から2025 年3月までの会計年度
-----------------
FY2024 Q1決算概要
取締役・CFO 川口 俊
2024年度第1四半期決算について、説明申し上げます。
本日は、決算説明のあとに企業価値向上プロジェクトについて前回からの補足説明をさせていただきます。
-----------------
© Ricoh Aug 6, 2024 3FY2024 Q1決算のポイント
業績
株主還元 自己株式取得 300億円は予定通り進捗2024年度見通しは変更なし
企業価値向上プロジェクトの費用・効果は機関決定に従い適時開示見通し増収減益、社内計画並み
企業価値向上プロジェク

## 2. embeddingとvector storeへの保存

In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import  Chroma
from uuid import uuid4

In [9]:
# Embeddingsのインスタンス化とvector storeへの登録
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma(
    collection_name="ricoh_collection",
    embedding_function=embeddings,
    persist_directory="./data/"
)
uuids = [str(uuid4()) for _ in range(len(all_splits))]

vector_store.add_documents(documents=all_splits, ids=uuids)

In [10]:
# query試し1
query = "北米事業の状況"
results = vector_store.similarity_search(query, k=3)

for res in results:
    print(res.page_content)
    print("-----------------")

© Ricoh Aug 6, 2024 1014 37 51 232 
37 19 35 56 264 
41 
33% -4% 9% 14% 10%ITインフラ ITサービス AS BPS WEFY2023 Q1 FY2024 Q1373 417 
FY2023 Q1 FY2024 Q1
リコーデジタルサービス オフィスサービス概況:米州
前年同期比
FY2024 Q1 
+11.8%
+44  売上高 (億円)
BPSは前年並み
オフサイト中心に好調
業務効率化とプライシングコントロールに取組み収益性改善
ワークプレイスエクスペリエンス（WE*2）、一部案件の期ずれ
Cenero*1とのシナジー拡大、米州既存顧客への導入すすむ
FY2023 Q4に事業の選択と集中に伴いeDiscovery事業売却分野別動向 (億円)
前年同期比
（除為替影響） (-16%) (-4%) (+0%) (-3%) (17%)*2一部商談の期ズレ等により弱含み。事業売却の一過性影響
*1 買収企業の位置付けはP.24を参照
*2 Workplace Experience/ ワークプレイスエクスペリエンス:コミュニケーションサービスより名称変更
米州のオフィスサービスです。
米州のBPSは、オフサイト中心に好調に推移しています。
なお、2023 年度までコミュニケーションサービスとしていた分野の名称を、より適切な表現にすべく 2024年度
からワークプレイスエクスペリエンスと名称を変更しました。
このワークプレイスエクスペリエンスでは、買収したCeneroが、大きなシナジーを生んでいます。
第１四半期において、計画していた一部大型案件の期ずれが発生しているものの、順調に進展しています。
「事業の選択と集中」に伴い、3月に、主に訴訟のプロセスを支援する事業のeDiscoveryの売却をしたこと
が業績（売上の減少）に影響しています。
-----------------
計画並み
© Ricoh Aug 6, 2024 1263 65 171 202 234 268 
FY2023 Q1 FY2024 Q1
-12 -3 
FY2023 Q1 FY2024 Q1リコーインダストリアルソリューションズ
FY2024 Q1 
+14.5%
＋8+33 
サーマル
産プロ売

In [11]:
# query試し2
query = "経営状況"
results = vector_store.similarity_search(query, k=3)

for res in results:
    print(res.page_content)
    print("-----------------")

© Ricoh Aug 6, 2024 18セグメント別 売上・営業利益
前年差FY2024
見通しFY2023
実績（単位:億円）
+941 19,470 18,528 売上 リコーデジタルサービス
+31 440 408 営業利益
+735 5,580 4,844 売上 リコーデジタル
プロダクツ
+116 290 173 営業利益
+218 2,840 2,621 売上 リコーグラフィック
コミュニケーションズ
+15 170 154 営業利益
+94 1,230 1,135 売上 リコーインダストリアル
ソリューションズ+68 65 -3 営業利益
-96 360 456 売上 その他
(カメラ、新規事業他)
+20 -85 -105 営業利益
-383 -4,480 -4,096 売上 全社・消去
-171 -180 -8 営業利益
+1,510 25,000 23,489 売上 合計
+79 700 620 営業利益期初見通しから変更なし
-----------------
© Ricoh Aug 6, 2024 28四半期営業利益推移
275 
210 207 
96 
-212 -93 8 
-156 56 74 126 143 96 138 161 391 
101 93 175 248 
63 5.8%
4.1%4.2%
1.9%
0.2%1.3%1.8%2.9%3.0%2.1%2.7%2.9%6.5%
1.9%1.6%3.0%3.8%
1.1%
Q1 Q2 Q3 Q4 Q1 Q2 Q3 Q4 Q1 Q2 Q3 Q4 Q1 Q2 Q3 Q4 Q1 Q2 Q3 Q4 Q1
FY19 FY20 FY21 FY22 FY23 FY24営業利益（億円） 営業利益率
(赤字) (赤字) (赤字)
-----------------
↑
RGC構造改革
© Ricoh Aug 6, 2024 5101要因別営業利益
63106 101【前年からの増減 】
FY2024 Q1 FY2023 Q1 FY2023 Q1
(一過性要因除く)体質強化販売ミックス
-26
+13
-43(単位:億円)
一過性要因RDS -31
RDP +3
RGC +20
RIS +5
その他 -23
+51為替影響
[+75] [-210] [-100] [+310] [-15] [

In [12]:
# retrieverの作成
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("アメリカ市場の動向")
for res in retrieved_docs:
    print(res.page_content)
    print("-----------------")

© Ricoh Aug 6, 2024 26FY2024 FY2023 売上高
前年同期比 Q1 Q4 Q3 Q2 Q1
+4% -8% +1% -5% -12% 日本
+24% +12% +6% +14% +12% 米州
+9% +0% +1% +9% +6% (為替影響除く）
+20% +11% +14% +11% +10% 欧州
+7% -2% +4% -1% +2% (為替影響除く）FY2024 FY2023 売上高
前年同期比 Q1 Q4 Q3 Q2 Q1
+1% +3% +1% -1% -0% 日本
-2% +1% +3% +7% +9% 米州
-14% -10% -1% +3% +3% (為替影響除く）
+4% +7% +9% +13% +14% 欧州
-7% -6% -2% -0% +5% (為替影響除く）データ編 リコーデジタルサービス・リコーグラフィックコミュニケーションズ
オフィスプリンティング
地域別（ハード+ノンハード合計）商用印刷
地域別（ハード+ノンハード合計）FY2024 FY2023 売上高
前年同期比 Q1 Q4 Q3 Q2 Q1
-5% -1% +5% +11% +16% ハード
-14% -10% -1% +5% +10% (為替影響除く）
+4% +6% +4% +1% +3% ノンハード
-3% -1% -0% -3% -1% (為替影響除く）FY2024 FY2023 売上高
前年同期比 Q1 Q4 Q3 Q2 Q1
+32% +7% +4% +16% +11% ハード
+18% -3% -1% +10% +5% (為替影響除く）
+16% +16% +10% +7% +9% ノンハード
+4% +5% +4% +1% +4% (為替影響除く）
-----------------
© Ricoh Aug 6, 2024 1014 37 51 232 
37 19 35 56 264 
41 
33% -4% 9% 14% 10%ITインフラ ITサービス AS BPS WEFY2023 Q1 FY2024 Q1373 417 
FY2023 Q1 FY2024 Q1
リコーデジタルサービス オフィスサービス概況:米州
前年同期比
FY2024 Q1 
+11.8%
+44  売上高 (億円)
BPSは前年並み
オフサ

## 3. promptの生成と回答

In [13]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [14]:
llm = ChatOpenAI(model="gpt-4o-mini")

system_prompt = (
    "You are an analyst of company financial data."
    "Use the following pieces of retrieved context to answer the user's question. "
    "If you don't know the answer, sya that you don't know.  Make consice sentences."
    "Question and retrieved context are Japanease. So answer in Japanese."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Q&A実施
question = "北米市場の概況を教えてください。"
response = rag_chain.invoke({"input": question})

def print_jp(response):
    sentence = ""
    for r in response["answer"]:
        sentence += r
        if r == "。":
            print(sentence)
            sentence = ""

print(question, '\n')
print_jp(response)

北米市場の概況を教えてください。 

北米市場では、FY2024の売上高が前年同期比で+24%の増加を見せています。
また、オフィスサービスのストック売上が18%増収し、利益成長に貢献しています。
ハードのオフィスプリンティングでは、A3MFP中心に受注残が発生しており、受注は堅調ですが、一部地域ではノンハードが想定を下回る結果となっています。


In [15]:
question = "企業買収状況について教えてください。"
response = rag_chain.invoke({"input": question})

print(question, '\n')
print_jp(response)

企業買収状況について教えてください。 

リコーは、2023年にアイルランドのPFHを買収し、収益に貢献しています。
また、natif.aiという会社もリコーに加わり、DocuWareとの組み合わせによる案件獲得が進んでいます。
買収企業全体で24%の増収を達成し、シナジー創出が進んでいると報告されています。


In [16]:
for document in response["context"]:
    print(document)
    print("----")

page_content='FY2024 Q1決算概要
取締役・CFO 川口 俊
2024年度第1四半期決算について、説明申し上げます。
本日は、決算説明のあとに企業価値向上プロジェクトについて前回からの補足説明をさせていただきます。' metadata={'page': 2, 'source': './data/finance_report.pdf'}
----
page_content='。
また、「一部地域でのノンハード減収」における一部地域はアメリカとなります。最大マーケットであるアメリカで
オフィスプリンティングの出力量が弱含み、想定を下回る実績となりました。
リコーデジタルプロダクツは、昨年度末にかけて販売在庫が整理された結果、生産調整が終了し、当年度は
順調に生産を進めております。こちらは、利益をしっかりと出しています。また、製品ミックス等も改善しています。
7月1日に東芝テック株式会社とのジョイントベンチャー、エトリア株式会社が発足しました。さまざまな施策の
展開を開始しています。
リコーグラフィックコミュニケーションズは、これまで着実に積み上げてきた市場稼働台数がノンハード売上につなが
り、増収増益となりました。
インダストリアルソリューションズでは、サーマル事業が回復を示し第 1四半期は、増収増益となり、
黒字化しました。
なお、2024 年度通期見通しに変更はありません。 5月の2023 年度通期決算時にもお伝えしましたが、
今後企業価値向上プロジェクトに関連して追加での費用・効果が発生する可能性があります。
こちらにつきましては必要な機関決定が行われ次第、皆様に公表いたします。
＜株主還元＞
2024年2月に発表しました300億円設定の自己株取得においては、順調に進捗しています。
７月末までに累計で約 250億円の買い付けを行いました。' metadata={'page': 3, 'source': './data/finance_report.pdf'}
----
page_content='© Ricoh Aug 6, 2024 9228 
147 
72 59 67 245 181 
88 63 74 
7% 23% 22% 7% 11%ITインフラ ITサービス AS BPS WEFY2023 Q1 FY2024 

### * promptをカスタマイズするとき

In [17]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [18]:
template = """You are an analyst of company financial data. Use the following pieces of 
retrieved context to answer the user's question. If you don't know the answer, 
say that you don't know.  Make consice sentences. Question and retrieved context 
are Japanease. So answer in Japanese.

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

question = "企業買収状況について教えてください。"

response = rag_chain.invoke(question)

In [22]:
print(question, '\n')
sentence = ""
for doc in response:
    sentence += doc
    if doc == "。":
        print(sentence)
        sentence = ""

企業買収状況について教えてください。 

リコーは、買収企業が全体で24%の増収を記録しています。
特に、アイルランドのPFHが収益に貢献しており、natif.aiとDocuWareの組み合わせによる案件獲得も進んでいます。
買収企業間のシナジー創出が進んでいます。
